# Prediction Models

Später dann zum zusammenführen  
•	VotingClassifier  
•	StackingClassifier   
•	BaggingClassifier  
•	AdaBoostClassifier

Lineare Modelle:  Marco  
•   LinearRegression/Ridge  
•	Lasso*    
•	ElasticNet*   
•	OrthogonalMatchingPursuit*  

Naive Bayes:  Patrick  
•	BernoulliNB  
•	ComplementNB  
•	GaussianNB  
•	MultinomialNB  

Nearest Neighbors:  Marco  
•	KNeighborsClassifier  
•	RadiusNeighborsClassifier   

Decision Trees:  Thomas  
•	DecisionTreeClassifier  
•	ExtraTreeClassifier  
•   LGBMClassifier       
•	RandomForestClassifier  

Support Vector Machines:  Patrick  
•	LinearSVC  
•	LinearSVR  
•	NuSVC  
•	NuSVR  
•	OneClassSVM  
•	SVC  
•	SVR  

Neural Networks:  Thomas  
•	MLPClassifier  
•	MLPRegressor  

In [30]:
import numpy as np
import pipeline as pipe

from src.pipelines.build_pipelines import CustomPipeline, get_best_steps
from src.features import build_features
from src.features.build_features import *
from src.data import configuration as config

## Baseline
For our baseline we use a dummy classifier

In [31]:
from sklearn.dummy import DummyClassifier

# create a dummy classifier that predicts the most frequent class
estimator = DummyClassifier(strategy='constant', constant=1)
baseline_pipe = CustomPipeline(get_best_steps(customEstimator=estimator), skip_feature_evaluation=True)
baseline_pipe.run()

loading data
preparing data
running pipeline
evaluating pipeline
    fit_time: 0.08106112480163574
    score_time: 0.019800806045532228
    test_accuracy: 0.5240650475901691
    test_f1-score: 0.22924001725660642
    test_mcc: 0.0
storing model and prediction


## Linear Models
Linear regression is used to predict a continuous variable. In our case we have a categorical variable with the values 1, 2 or 3. Therefore linear regression is not suitable.

Logistic Regression is a Machine Learning classification algorithm that is used to predict the probability of a categorical dependent variable. In the following we try out this model

In [32]:
from sklearn.linear_model import LogisticRegression

# create an instance of the model
estimator = LogisticRegression(multi_class='auto')

testpip = CustomPipeline(get_best_steps(customEstimator=estimator))
testpip.run()

loading data
preparing data
running pipeline
evaluating pipeline
    fit_time: 0.8765987873077392
    score_time: 0.024641036987304688
    test_accuracy: 0.5395445695991647
    test_f1-score: 0.34493203720478044
    test_mcc: 0.0786119082964518
storing model and prediction


An accuracy of 0.539 and an F1-score of 0.345 indicate that the model is not able to make accurate predictions, and an MCC of 0.078 suggests that the model's performance is only slightly better than random guessing.

In [33]:
# try out with only the numerical values
estimator = LogisticRegression(multi_class='auto')

feature_remover = RemoveFeatureTransformer(config.categorical_columns)

testpip = CustomPipeline([
    ('feature_remover', feature_remover),
    ('estimator', estimator)
])
testpip.run()

loading data
preparing data
running pipeline
evaluating pipeline
    fit_time: 0.5401473999023437
    score_time: 0.008696603775024413
    test_accuracy: 0.5445102481352101
    test_f1-score: 0.34635715199848016
    test_mcc: 0.08864539493241383
storing model and prediction


Dropping the categorical columns improved the score only very slightly. We conclude that using a linear model is not suitable for our data task.

## Nearest Neighboor

In [39]:
from sklearn.neighbors import KNeighborsClassifier

# trains and predicts on the transformed data

bestScore = {}
bestK = 0
for k in range(2,10):    
    estimator = KNeighborsClassifier(n_neighbors=k)
    testpip = CustomPipeline(
        get_best_steps(customEstimator=estimator),
        skip_evaluation=False,
        print_evaluation=False,
        skip_storing_prediction=True)
    testpip.run()
    
    if len(bestScore) <= 0:
        bestScore = testpip.evaluation_scoring
        bestK = k
        
    elif bestScore['test_mcc'].mean() < testpip.evaluation_scoring['test_mcc'].mean():
        bestScore = testpip.evaluation_scoring
        bestK = k

print(f'The best score was achieved with k of {bestK}:')
for score in bestScore:
    print('    ' + score + ':', bestScore[score].mean())

loading data
preparing data
running pipeline
evaluating pipeline
loading data
preparing data
running pipeline
evaluating pipeline
loading data
preparing data
running pipeline
evaluating pipeline
loading data
preparing data
running pipeline
evaluating pipeline
loading data
preparing data
running pipeline
evaluating pipeline
loading data
preparing data
running pipeline
evaluating pipeline
loading data
preparing data
running pipeline
evaluating pipeline
loading data
preparing data
running pipeline
evaluating pipeline
The best score was achieved with k of 5:
    fit_time: 0.16252760887145995
    score_time: 1.1579453468322753
    test_accuracy: 0.6698922589734975
    test_f1-score: 0.5677577641385602
    test_mcc: 0.3612988871356594
